In [172]:
bp = '0'

def glue(els):
    return '|'.join(els)

def encaps(s):
    return '(' + s + ')'

def unglue(s):
    return s.split('|')

def suffixes(s):
    """ proper suffixes """
    
    return [s[i:] for i in range(1, len(s))]

def prefixes(s):
    """ proper prefixes """
    return [s[:i] for i in range(1, len(s))]

print(prefixes('21'))
print(suffixes('21'))

['2']
['1']


In [190]:
def is_suffix(suffix,s):
    return suffix in suffixes(s) # could use endswith

def is_prefix(prefix,s):
    return prefix in prefixes(s) # could use startswith

s = '_|1_'

def opteps(s):
    """ Optional null-move re-phrasing: _|1_ -> 1? """

    els = unglue(s)

    if len(els) > 1 and '_' in els:
        optionals = []
        for el in els:
            if el != '_':

                if is_suffix('_',el):
                    el = el[:-1]

                if len(el) > 1:
                    el = encaps(el)

                el += '?'
                optionals.append(el)

        return glue(optionals)
    else:
        return s

def optpre(s):
    """ Optional prefixes re-phrasing: 02|2 -> 0?2 """

    els = unglue(s)

    new_forms = []

    drop = []

    # consider any element as a potential prefix
    for i, suf in enumerate(els):

        # store any matching cases
        matches = []

        for j, word in enumerate(els):

            if is_suffix(suf,word):

                matches.append(word)

        if matches:
            # post-processing

            pres = [x.replace(suf,'') for x in matches]
            pre = glue(pres)

            if len(pre) > 1:
                pre = encaps(pre)

            new_form = pre+'?'+suf

            drop.append(suf)
            drop.extend(matches)

            new_forms.append(new_form)

    unchanged = [x for x in els if x not in drop]
    res = glue(new_forms + unchanged)

    return res
    
def optsuf(s):
    """ Optional prefixes re-phrasing: 02|021|022 -> 02(1|2)? """
    
    els = unglue(s)

    new_forms = []

    drop = []

    # consider any element as a potential prefix
    for i, pre in enumerate(els):
        
        # store any matching cases
        matches = []

        for j, word in enumerate(els):

            if is_prefix(pre,word):

                matches.append(word)

        if matches:
            # post-processing

            suffixes = [x.replace(pre,'') for x in matches]
            suffix = glue(suffixes)

            if len(suffix) > 1:
                suffix = encaps(suffix)

            new_form = pre+suffix+'?'

            drop.append(pre)
            drop.extend(matches)

            new_forms.append(new_form)
    
    unchanged = [x for x in els if x not in drop]
    res = glue(new_forms + unchanged)
    
    return res

def opt(s):
    
    res = optpre(s)
    res = optsuf(res)
    
    return res

def simplify(graph, bp):
    """ Simplify a graph by eliminating bypassable state bp. 
    
    Aka Méthode de Brzozowski et McCluskey
    """

    starts = [src for (src, dst) in graph if dst == bp and src != dst]
    ends = [dst for (src, dst) in graph if src == bp  and src != dst]
    cycles = [move for (src,dst), move in graph.items() if src == dst == bp]

    # this is broken
    if cycles:
        cycling_move = encaps(glue(cycles)) + '*'
    else:
        cycling_move = ''

    new_graph = {}

    for src in starts:
        for dst in ends:

            move_in = graph[(src,bp)]
            move_out = graph[(bp,dst)]
            
            if '|' in move_in:
                move_in = encaps(move_in)
            
            if '|' in move_out:
                move_out = encaps(move_out)
                       
            move_through = move_in + cycling_move + move_out

            new_graph[(src,dst)] = move_through

    for (src, dst), move in graph.items():

        if (src is not bp) and (dst is not bp):

            k = (src,dst)

            if k in new_graph:
                new_graph[k] += '|' + graph[k]
            else:
                new_graph[(src,dst)] = move
    
    return new_graph

In [197]:
graph = {
    ('S','F'):'_',
    ('S','0'):'0',
    ('S','1'):'1',
    ('S','2'):'2',
    ('0','1'):'1',
    ('0','2'):'2',
    ('0','F'):'_',
    ('1','2'):'2',
    ('1','0'):'0',
    ('1','F'):'_',
    ('2','1'):'1',
    ('2','0'):'0',
    ('2','F'):'_',
}

alphabet = ['0','1','2','_']
nodes = ['S','0','1','2','F']

graph = simplify(graph, '0')

graph = {k: optpre(opteps(v)) for k,v in graph.items()}

graph

{('1', '1'): '01',
 ('1', '2'): '0?2',
 ('1', 'F'): '0?',
 ('2', '1'): '0?1',
 ('2', '2'): '02',
 ('2', 'F'): '0?',
 ('S', '1'): '0?1',
 ('S', '2'): '0?2',
 ('S', 'F'): '0?'}

In [199]:
graph = simplify(graph, '2')

graph 

{('1', '1'): '0?2(02)*0?1|01',
 ('1', 'F'): '0?2(02)*0?|0?',
 ('S', '1'): '0?2(02)*0?1|0?1',
 ('S', 'F'): '0?2(02)*0?|0?'}

In [195]:
graph = simplify(graph, '1')

graph

{('S', 'F'): '(0?2(02)*0?1|0?1)(0?2(02)*0?1|01)*(0?2(02)*0?|0?)|0?2(02)*0?|0?'}